# Build Segment-Based Library

With hydro-conditioned DEM, we will identify streams, divide them into segments and create a library for each segment.

## Import python packages

In [1]:
import os

# import fldpln_py package
import fldpln_py
import matlab

# import the model module from the fldpln package
from fldpln.model import *

## Create a FLDPLN Model Object

Before we can use the FLDPLN model, we must first create a FLDPLN model object which automatically initialize the model Python package. This will activate the MATLAB Runtime and will take a littlle bit of time.

In [2]:
fld = FLDPLN()

Initialize FLDPLN model python package ...


## Generate Segments

The first step to create a FLDPLN library is to identify stream networks and create stream segments from a hydro-conditioned digital elevation model (DEM). The stream networks/pixels are first identified using a flow accumulation threshold ('strfac'). The stream networks are first divided into reaches (stream links in ArcGIS' term), which are stream segments between headwater and confluence pixels or between two consecutive confluence pixels. Note that pixels flow out of the DEM are treated as confluence pixels in this process. Those natural reaches/segments are further divided based on flow accumulation jumps greater than or equal to 'segfac'. Then the segments are bisected until all segments are no longer than 'seglen'. Note that the units for flow accumulation are squared miles and the unit for segment length is miles.

The default values for the 3 parameters are: 70 sq. miles, 25 sq. miles, and 2 miles for the libraries in KS. Note that Kansas FLDPLN libraries stream networks are more coarse grained than the National Water Model (NWM) reaches. This is because the stream networks are generated just to use available USGS gauges. If the NWM discharge is used for the flood inundation mapping, the stream networks need to be densified.

In [3]:
# Wildcat example
bildir = 'E:/fldpln/sites/wildcat_10m_3dep/bil'
segdir = 'E:/fldpln/sites/wildcat_10m_3dep/segs_fldpln'
# Verdigris example
# bildir = 'E:/fldpln/sites/verdigris_10m/bil'
# segdir = 'E:/fldpln/sites/verdigris_10m/segs'

# Input flow direction and accumulation BIL files
fdrf = os.path.join(bildir,'fdr.bil')
facf = os.path.join(bildir,'fac.bil')

# segment parameters
strfac = 50; # flow accumulation threshold (in sq. miles) for identifying stream networks. 50 for Wildcat to include the upstream gauge. 70 for Verdigris and most KS watersheds
segfac = 25; # flow accumulation threshld (in sq. miles) for segment stream networks. 25 is the default in KS
seglen = 2; # segment length in miles. usually is the SQRT of sgefac. 2 for Wildcat and 5 for Verdigris and others in KS

# generate segments
fld.GenerateSegments(fdrf, facf, strfac, segfac, seglen, segdir)

The FSP and segment information can be also saved as two CSV files for generating segment shapefile for selecting a subset of segments for build library.

In [4]:
# write FSP and segment info CSV files for creating segment shapefile
seg_list = []; # for all the segments
# seg_list = [1;2;3]; # column vector for selected segments
fld.WriteSegmentFspCsvFiles(bildir, segdir, seg_list, segdir)

## Select Segments and Create Spatial Mask

We don't need to generated FLDPLN library for every segment identified. And typically we want to exclude large waterbodies such as reservoirs/lakes and the segments that within them from the modeling.

### Generate Segment Shapefile 

Here we create a segment shapefile using the ArcGIS Pro's Stream to Features tool with input str_segid.bil and fdr.bil as the input rasters. The segment shapefiles can then be used to select a subset of segments for running the FLDPLN model.

Note that the Raster to Polyline tool doesn't work properly in some cases. In the figure below, the left stream polyline is generated by the Raster to Polyline tool, and the right stream polyline is generated by the Stream to Feature tool. The left polyline is erroneous. Also note that the Stream to Features tool connects upstream segments at the downstream confluence pixel.

![](./stream2feature.PNG)

Using the shapefiles generated in GIS, we can take a close look on the segments. We may delete some segments in large waterbodies (e.g., lakes) and just select a subset of segment to build their library. 

Currently, the selected segments can be exported as a shapefile in GIS. The shapefile is used as one of the inputs to build segment library. 

### Create Spatial Mask

For large waterbodies such as reservoirs and lakes, we want to mask them out so that they can be handled differently. Here we can use GIS software to generate a mask raster in BIL format that masks out all the reserviors and lakes within the DEM. When such a mask raster is provided, those cells are excluded from the FPPs and any FSPs within the mask is also remove from the modeling. 

The mask BIL file is used as one of the inputs to build segment library.

**Note that this has NOT been implemented yet!**

## Create Segment Library

With the segments identified, we can create the library all of them or a subset of selected segments by. In addition, we can also use a raster mask.

**Note that this is the most time consuming step in building FLDPLN libraries, especially the max stage modelded is high.**

### Create a library for selected segments

With the segment shapefiles, we can select the subset of segments for the Wildcat Creek (sgement IDs from 1 to 16) to create a library. The selected segments are provided as a shapefile. The segment ID field/column in the shapefile need to be specified.  All the selected segments by default have the same max. flood stage as specified by model parameter mxht. We can also specify a second column in the shapefile to define different max. flood stages for individual segment.

In [ ]:
# set input and output folders
# Wildcat example
bildir = 'E:/fldpln/sites/wildcat_10m_3dep/bil' 
segdir = 'E:/fldpln/sites/wildcat_10m_3dep/segs_fldpln'
libdir = 'E:/fldpln/sites/wildcat_10m_3dep/rawlib_fldpln' # raw sgement-based library

# Applying spatial mask if provided. The masked filled DEM removes waterbodies to prevent flooding them.
# In the future, only a mask raster BIL is need which will be applied to filled DEM instead of a masked filled DEM that's currently used!
filmskfile = '' # no spatial mask for Wildcat
# filmskfile = 'E:/fldpln/sites/verdigris_10m/bil/fil_masked.bil'

# Select a subset of segments or remove segments within waterbodies before creating a FLDPLN library
# The subset of segments is defined using a shapefile.
# segshpfile = {'file': ''} # all the segments will be used and they use the fldmx
segshpfile = {'file':'E:/fldpln/sites/wildcat_10m_3dep/segment_shapefiles/wildcat_segments.shp'} # wildcat creek segments
# segshpfile = {'file':'E:/fldpln/sites/verdigris_10m/segs_5mi/dam_break_segments.shp'} # verdigris

# Additional attributes if a sgement shapefile is provided
segshpfile['segid_field'] = 'grid_code'
segshpfile['seg_fldmx_field'] = '' # set to '' when all the segments use the fldmx. Otherwise specify a field in the shapefile

# Set FLDPLN model parameters
fldmn = 0.01 # typically set to 1 centermeter or 0.0328084 foot DEM's vertical unit
fldmx = 15 # max. stage modeled. wildcat_10m_3dep and verdigris DEM vertical unit is in meters
dh = 1 # vertical step size in DEM's vertical unit
mxht = 0 # max(dem+flood height) to cease flooding. enter 0 for no cap height

# FLDPLN model efficiency parameters
# model type: choose one from {'hd', 'ram0', 'ram'}. 
mtype = 'ram' # choose either 'ram' (machine has RAM >= 64G) or 'hd' (uses least RAM)
# parallelization setting: 'none', 'parfor', 'parfeval'; recommend either 'parfeval'
para = {'type': 'parfeval'}
# Can also set the number of cores. Will use max. cores if the attribute is NOT set
para['numcores'] = 6 # my office computer has 8 core. Comment out to use all the cores available
para['worker_type'] = 'Processes' # 'Threads' is not supported by compiled Python package!

# Create segment library
fld.CreateSegmentLibrary(bildir, segdir, filmskfile, segshpfile, fldmn, fldmx, dh, mxht, libdir, mtype, para)

### Work with Existing Library

If segment libraries already exist and if the specified fldmx is lower than the existing library’s max. stage, the model simply generates the new library with existing information and DOES NOT re-run the model. 

If the specified fldmx is higher than the existing library’s max. stage, the model continues from the existing library without starting from the scratch.

Note that the model creates the new segment libraries  UNDER THE SAME FOLDER where the existing libraries reside.

In [ ]:
# need example code

## Format Segment Library for Tiling

In [ ]:
# BIL file directory
bildir = 'E:/fldpln/sites/wildcat_10m_3dep/bil' # Wildcat
# bildir = 'E:/fldpln/sites/verdigris_10m/bil'
# segment file directory
segdir = 'E:/fldpln/sites/wildcat_10m_3dep/segs_fldpln'
# segdir = 'E:/fldpln/sites/verdigris_10m/segs'

# raw segment library dir
libdir = 'E:/fldpln/sites/wildcat_10m_3dep/rawlib_fldpln'
# libdir = 'E:/fldpln/sites/verdigris_10m/seglib_fldsensing'

# Outputs:
# Output library folder
dirout = 'E:/fldpln/sites/wildcat_10m_3dep/seglib_fldpln' # reformatted library for tiling and mapping
# dirout = 'E:/fldpln/sites/verdigris_10m/lib_fldsensing'

# Format raw segment library
fld.FormatSegmentLibrary(bildir, segdir, libdir, dirout)

## Assign Stream Order to Segments

Stream orders are used while interpolating the depth of flow (DOF) at FSPs where low order streams are handled before high order streams. The general rules for assigning stream order are:
* A stream (or reach) consists of several connected segments, which may meet with other streams at confluences
* Main streams have low orders than tributary streams
* Order must be unique to each stream in a library. No two streams can have the same order even if the streams are in different sub-watersheds in the library

The above requirements are necessary to make sure the interpolation on DOF can be carried out correctly.

In [ ]:
# code to automatically generate stream orders for segments

## Terminate the FLDPLN Model

This is done by delete the FLDPLN model object.

In [5]:
del fld

## Issues and Questions

* Messages printed out (disp() or fprintf()) to the stdout in the FLDPLN Python package CANNOT be printed in this notebook. Running python scripts (such as wildcat_segments.py, wildcat_library.py and wildcat_format_library.py) works just fine.
